<a href="https://colab.research.google.com/github/edcalderin/LLM_Tech/blob/master/building_a_rag_chain_using_langchain_expression_language_lcel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a RAG chain using LangChain Expression Language (LCEL)

https://medium.com/data-science/building-a-rag-chain-using-langchain-expression-language-lcel-3688260cad05

In [1]:
!pip install -qU langchain-community langchain-core langchain-openai langchain-text-splitters beautifulsoup4 faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

## Indexing

In [4]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large", api_key=OPENAI_API_KEY)

In [5]:
import bs4
import faiss
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

page_url = "https://en.wikipedia.org/wiki/Marie_Curie"
bs4_strainer = bs4.SoupStrainer(attrs={"class": "mw-body-content"})
loader = WebBaseLoader(web_paths=[page_url], bs_kwargs={"parse_only": bs4_strainer})
docs = []
for doc in loader.lazy_load():
    docs.append(doc)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False
)

texts = text_splitter.split_documents(docs)

print(len(texts))

1084


In [6]:
vector_store = FAISS.from_documents(texts, embeddings)
retriever = vector_store.as_retriever()

In [7]:
response = retriever.invoke("who was marie curie?")
response

[Document(id='d90e2b86-d4fb-43f5-ae08-ccb41105334b', metadata={'source': 'https://en.wikipedia.org/wiki/Marie_Curie'}, page_content='Marie Curie was the first woman to win a Nobel Prize, the first person to win two Nobel Prizes, the'),
 Document(id='ead2ce17-5afc-4e25-a60e-306d65ff50ee', metadata={'source': 'https://en.wikipedia.org/wiki/Marie_Curie'}, page_content='Curie (/ˈkjʊəri/ KURE-ee;[1] French: [maʁi kyʁi]), was a Polish and naturalised-French physicist'),
 Document(id='5683d120-c5ca-43fc-9f50-879ec091b2a5', metadata={'source': 'https://en.wikipedia.org/wiki/Marie_Curie'}, page_content='ⓘ; née\xa0Skłodowska; 7 November 1867 – 4 July 1934), known simply as Marie Curie (/ˈkjʊəri/'),
 Document(id='b8272d90-97c9-41c8-b439-a61ff4eac54d', metadata={'source': 'https://en.wikipedia.org/wiki/Marie_Curie'}, page_content='Marie CurieCurie, c.\u20091920BornMaria Salomea Skłodowska(1867-11-07)7 November 1867Warsaw, Poland,')]

## Retrieval and Generation

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from operator import itemgetter

In [64]:
prompt: str = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
"""

prompt_template = ChatPromptTemplate.from_template(prompt)

llm = ChatOpenAI(model="gpt-4o", api_key=OPENAI_API_KEY)

def format_docs(docs: list)->str:
    return "\n\n".join([doc.page_content for doc in docs])

chain = (
    RunnableParallel(
        question = RunnablePassthrough(),
        context = itemgetter("question") | retriever | format_docs
    )
    | prompt_template
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "who was marie curie?"})

'Marie Curie was a Polish and naturalized-French physicist known for her groundbreaking research on radioactivity. She was the first woman to win a Nobel Prize and the first person to win two Nobel Prizes. Curie was born on November 7, 1867, in Warsaw, Poland.'

## Self Evaluation (I)

In [18]:
from enum import Enum
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class GradeEnum(Enum):
    CORRECT = "correct"
    INCORRECT = "incorrect"

class LLMEvalResult(BaseModel):
    grade_enum: GradeEnum = Field(description="Final grade label. Accepted labels: correct, incorrect")
    description: str = Field(description="Explanation about why the specific grade was accepted. Must be concise. No more of 2 sentences")

json_output_parser = JsonOutputParser(pydantic_object=LLMEvalResult)


'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"GradeEnum": {"enum": ["correct", "incorrect"], "title": "GradeEnum", "type": "string"}}, "properties": {"grade_enum": {"$ref": "#/$defs/GradeEnum", "description": "Final grade label. Accepted labels: correct, incorrect"}, "description": {"description": "Explanation about why the specific grade was accepted. Must be concise. No more of 2 sentences", "title": "Description", "type": "string"}}, "required": ["grade_enum", "description"]}\n```'

In [58]:
eval_prompt: str = """
You are a teacher evaluating a test.
You are provided with a question along with an answer for the question written
by a student. Evaluate the question-answer pair and provide feedback.

{format_instructions}

Question : {question}
Answer : {answer}
"""

eval_prompt_template = ChatPromptTemplate.from_template(eval_prompt)

# Step 1: Retrieve context
step_1 = {
    "question": RunnablePassthrough(),
    "context": itemgetter("question") | retriever | format_docs,
}

# Step 2: Generate student's answer from context + question
step_2 = RunnableParallel(
    answer = prompt_template | llm | StrOutputParser(),
    question = itemgetter("question"),
    format_instructions = lambda _: json_output_parser.get_format_instructions()
)

# Step 3: Evaluate answer using eval_prompt_template
step_3 = eval_prompt_template | llm | json_output_parser

# Combine the steps
full_chain = step_1 | step_2 | step_3
full_chain.invoke({"question": "Who was Marie Curie?"})

{'grade_enum': 'correct',
 'description': "The answer accurately describes Marie Curie's identity, her scientific contributions, and her achievements, including her Nobel Prizes."}

## Self Evaluation (II)

In [73]:
eval_prompt_with_context: str = """
You are a teacher evaluating a test.
You are provided with a question along with an answer for the question written
by a student. Evaluate the question-answer pair using the provided context and
provide feedback. Only mark the answer as correct if it agress with the
provided context

{format_instructions}

Context : {context}
Question : {question}
Answer : {answer}
"""

eval_prompt_context_template = ChatPromptTemplate.from_template(eval_prompt_with_context)

full_chain = (
    RunnableParallel(
        question = RunnablePassthrough(),
        context = itemgetter("question") | retriever | format_docs,
    )
    | RunnableParallel(
        answer = prompt_template | llm | StrOutputParser(),
        question = itemgetter("question"),
        context = itemgetter("context"),
        format_instructions = lambda _: json_output_parser.get_format_instructions()
    )
    | RunnableParallel(input=eval_prompt_context_template | llm | json_output_parser, context = itemgetter("context"), answer = itemgetter("answer"))
)

full_chain.invoke({"question": "What did to Marie Curie so famous?"})

{'input': {'grade_enum': 'correct',
  'description': "The answer agrees with the context, highlighting Marie Curie's achievements with Nobel Prizes."},
 'context': 'Marie Curie was the first woman to win a Nobel Prize, the first person to win two Nobel Prizes, the\n\nin a Paris street accident. Marie won the 1911 Nobel Prize in Chemistry for her discovery of the\n\n^ Her 1911 Nobel Prize in Chemistry was granted to "Marie Sklodowska Curie" File:Marie\n\nMarie Curie on Nobelprize.org \nvte Marie and  Pierre CurieDiscoveries\nCurie\'s law\nCurie–Weiss law',
 'answer': 'Marie Curie became famous for being the first woman to win a Nobel Prize and the first person to win two Nobel Prizes. She won the Nobel Prize in Physics in 1903 and the Nobel Prize in Chemistry in 1911 for her discoveries in radioactivity. Her groundbreaking work in these fields significantly advanced scientific understanding.'}